In [1]:
import sys
sys.path.append('..')

In [2]:
from abc import ABC, abstractmethod
from random import randint

## Optimization problems and algorithms

### Optimization problem

An **optimization problem** is a problem that involves finding the best solution from a set of possible solutions, according to some criteria or objective. The goal is to either **maximize** or **minimize** a particular quantity (called the **objective function**) while satisfying certain constraints.

**A solution to an optimization problem is problem dependent.**

### Optimization algorithm

An **optimization algorithm** is a method used to find the best solution to an optimization problem. It is an interative algorithm that is able to return a solution at each iteration. The goal is to navigate the solution space, explore potential solutions, and ultimately identify the optimal or near-optimal solution according to the objective function.

**An optimization algorithm is not problem dependent.** You can use the exact same algorithm to solve any optimization problem, as long as you know how you can navigate the solution space.

### Solution

Let's start by implementig a generic `Solution` class.

What should a generic solution to an optimization problem look like?

Given that a solution to a problem `A` is different from a solution to a problem `B`, it is hard to characterize this generic solution. But there are some things that are common to any optimization problem solution.

Let's think of attributes and methods that are common to any optimization problem.

A solution must have the following attributes and methods:
- representation: How the solution is enconded. Having a representation makes it possible for an algorithm to manipulate and evaluate the solution
- fitness(): The function that determines how good a solution is
- random_initial_representation(): If no representation is defined for the solution, it has to be possible to initialize it randomly.
    - **NOTE:** In class we called this method `random_initial_value` but `random_initial_representation` is more explicit. Let's use this name moving forward. The implementation in the library also adopts this name.


![Solution Class](images/solution.png)

Since these are problem-dependent, we can not implement them now. However, we can **enforce their implementation** in any subclass by defining this class as an **abstract class** with abstract methods. To do this, the class must inherit from `ABC`. Abstract methods have no implementation in the abstract class itself, but any subclass must implement them to allow object instantiation.

In [3]:
class Solution(ABC):
    def __init__(self, repr=None):
        # To initialize a solution we need to know it's representation.
        # If no representation is given, a representation is randomly initialized.
        if repr == None:
            repr = self.random_initial_representation()
        # Attributes
        self.repr = repr

    # Method that is called when we run print(object of the class)
    def __repr__(self):
        return str(self.repr)

    # Other methods that must be implemented in subclasses
    @abstractmethod
    def fitness(self):
        pass

    @abstractmethod
    def random_initial_representation():
        pass


The final implementation is available in `library/solution.py`

## Hill Climbing

Hill Climbing is one of the most intuitive and immediate techniques for solving optimization problems. It works by iteratively improving fitness in a stepwise refinement process, using the concept of neighborhood to explore potential solutions.

### Pseudo-code

1. Initialize current solution (usually at random)
2. Repeat
   1. Get neighbors of current solution
   2. Find best neighbor
   3. If best neighbor is better or equal than current solution, replace current solution by best neighbor
   4. If current solution hasn't changed, break the cycle
3. Return current solution

### Algorithm Implementation

Let's implement this algorithm using python. The function that implements the algorithm should receive the following arguments:
- `initial solution`: an instance of a solution to an optimization problem
- `maximization`: boolean that indicates if we're solving a maximization or minimization problem
- `max_iter`: maximum number of interations. By default should be very big.

In [4]:
from copy import deepcopy

def hill_climbing(initial_solution: Solution, maximization=False, max_iter=99999, verbose=False):
    current = initial_solution
    improved = True
    iter = 1

    while improved:
        if verbose:
            print(f'Current solution: {current} with fitness {current.fitness()}')

        improved = False
        neighbors = current.get_neighbors()

        for neighbor in neighbors:

            if verbose:
                print(f'Neighbor: {neighbor} with fitness {neighbor.fitness()}')

            if maximization and (neighbor.fitness() >= current.fitness()):
                current = deepcopy(neighbor)
                improved = True
            elif not maximization and (neighbor.fitness() <= current.fitness()):
                current = deepcopy(neighbor)
                improved = True
        
        iter += 1
        if iter == max_iter:
            break
    
    return current

Notice that we assume that a solution has the following methods:
- `fitness()`
- `get_neighbors()`

Additionally, `get_neighbors()` must return a list of solutions that also implement these methods.

The final implementation is available in `library/algorithms/hill_climbing.py`

## IntBin Optimization Problem

**Description:** The IntBin problem consists of finding the integer with greatest number of 1's in its binary representation

**Search space:** Integers from 1 to 15.

**Representation:** Binary string of 4 digits representing the integer.

**Fitness function:** f(x)= Number of 1's in binary representation of x

**Goal:** Maximize f(x).

### IntBin Solution

Using the previously defined generic `Solution`, we can now define the `IntBinSolution` class that implements the fitness and random intial representation methods for the IntBin problem.

This class represents a solution to the IntBin problem and **does not include any implementation related to the optimization algorithm that will be used to solve it**.

![IntBin Solution](images/intbin-solution.png)

In [5]:
class IntBinSolution(Solution):
    # The constructor can be ommited here since it only calls the super class constructor
    def __init__(self, repr=None):
        super().__init__(repr=repr)

    # Override the superclass's methods
    def random_initial_representation(self):
        # Generate random integer between 1 and 15
        random_n = randint(1, 15)
        # Transform it into its binary string representation with 4 digits
        return str(format(random_n, '04b'))

    def fitness(self):
        return self.repr.count('1')

Let's test it.

In [6]:
# Intialize with representation
solution = IntBinSolution('0001')

# Initialize with random representation
solution_random = IntBinSolution()

print(f'Solution {solution} with fitness {solution.fitness()}')
print(f'Random solution {solution_random} with fitness {solution_random.fitness()}')

Solution 0001 with fitness 1
Random solution 1101 with fitness 3


### Solving IntBin with Hill Climbing

To solve the IntBin problem using Hill Climbing, we need to define how to navigate the solution space. In hill climbing, the search space is navigated with the concept of neighborhood.

The algorithm requires the solution to have a `get_neighbors()` method. Therefore, we can **extend** the `IntBinSolution` and create a `IntBinHillClimbingSolution` that implements the `get_neighbors()` method.

![IntBin Hill Climbing Solution Inheritance](images/intbin-hillclimbing-solution.png)

There are two options to get the neighbors of a solution:
- Option 1 - Integer neighborhood: Each integer x has at most two neighbors: x-1 and x+1, except for boundaries (1 and 15).
- Option 2 - Bit flip neighborhood: Each binary representation of an integer x has as neighbors any other binary with a bit flipped.

#### Option 1 - Integer neighborhood: Each integer x has at most two neighbors: x-1 and x+1, except for boundaries (1 and 15).

Let's create a `IntBin_IntNeighborhood_HillClimbingSolution` class that inherits from `IntBinSolution` and implements the `get_neighbors()` method.

We also need to make sure that this method return a list of IntBin solutions that also implement `get_neighbors()`, meaning it should return a list of `IntBin_IntNeighborhood_HillClimbingSolution` instances.

In [11]:
class IntBin_IntNeighborhood_HillClimbingSolution(IntBinSolution):
    # This method must return a list of IntBin_IntNeighborhood_HillClimbingSolution objects (the neighbors)
    def get_neighbors(self):
        # Convert binary string representation into integer
        int_repr = int(self.repr, 2)

        if int_repr == 1:
            return [IntBin_IntNeighborhood_HillClimbingSolution(repr=format(2, '04b'))]
        elif int_repr == 15:
            return [IntBin_IntNeighborhood_HillClimbingSolution(repr=format(14, '04b'))]
        else:
            return [
                IntBin_IntNeighborhood_HillClimbingSolution(repr=format(int_repr-1, '04b')),
                IntBin_IntNeighborhood_HillClimbingSolution(repr=format(int_repr+1, '04b'))
            ]

Let's test it

In [8]:
# Initialize a random solution
solution = IntBin_IntNeighborhood_HillClimbingSolution('1010')
print('Solution', solution)

neighbors = solution.get_neighbors()
print('Neighbors:')
for neighbor in neighbors:
    print(neighbor)

Solution 1010
Neighbors:
1001
1011


We can now apply the HillClimbing algorithm to the IntBin problem by passing it an initial solution

In [12]:
# Randomly initialize a solution
initial_solution = IntBin_IntNeighborhood_HillClimbingSolution()

best_solution = hill_climbing(initial_solution, maximization=True, verbose=True)

print('Best solution:', best_solution)

Current solution: 1010 with fitness 2
Neighbor: 1001 with fitness 2
Neighbor: 1011 with fitness 3
Current solution: 1011 with fitness 3
Neighbor: 1010 with fitness 2
Neighbor: 1100 with fitness 2
Best solution: 1011


Let's see if the final solution changes with multiple runs

In [16]:
for _ in range(10):
    initial_solution = IntBin_IntNeighborhood_HillClimbingSolution()
    print('Final solution:', hill_climbing(initial_solution, maximization=True))

Final solution: 1011
Final solution: 0111
Final solution: 0011
Final solution: 0111
Final solution: 1011
Final solution: 0111
Final solution: 1011
Final solution: 0111
Final solution: 1111
Final solution: 1011


Different runs produce different solutions, and not always the global optimum (1111) is found

In the next notebook, we will implement the IntBin problem using Hill Climbing, adopting Option 2 (Bit Flip Neighborhood) to explore the solution space and analyse the differences.